This code is copied from https://github.com/MicrosoftLearning/mslearn-azure-ml/blob/main/Labs/06/Classification%20with%20Automated%20Machine%20Learning.ipynb 


# **Change the compute instance/cluster name according to your setup**

# Train a classification model with Automated Machine Learning

## Environment

Please select the appropriate kernel from Top right corner.
Here we use Python 3.8 Pytourch and Tensorflow kernel

In [1]:
!conda list

# packages in environment at /anaconda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
applicationinsights       0.11.10                  pypi_0    pypi
boltons                   23.0.0          py310h06a4308_0  
brotlipy                  0.7.0           py310h7f8727e_1002  
bzip2                     1.0.8                h7b6447c_0  
ca-certificates           2023.5.7             hbcca054_0    conda-forge
certifi                   2023.5.7           pyhd8ed1ab_0    conda-forge
cffi                      1.15.1          py310h5eee18b_3  
charset-normalizer        2.0.4              pyhd3eb1b0_0  
conda                     23.5.0          py310hff52083_1    conda-forge
conda-content-trust       0.1.3           py310h06a4308_0  
conda-package-handling    2.1.0           py310h06a4308_0  
conda-package-streaming   0.8.0           py310h06a4308_0  
cry

Before you start
You'll need the latest version of the azureml-ai-ml package to run the code in this notebook. Run the cell below to verify that it is installed.

```
pip install azure-ai-ml

```
Note: If the azure-ai-ml package is not installed, run 
```
pip install azure-ai-ml 

```
from terminal to install it.

Please activate appropriate conda environment into command line likewise
```
conda activate azureml_py38_PT_TF
```

You can also install it from this notebook

In [13]:
pip install azure-ai-ml

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.15.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mltable

Note: you may need to restart the kernel to use updated packages.


### Restart the kernel

In [1]:
!conda list

# packages in environment at /anaconda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
applicationinsights       0.11.10                  pypi_0    pypi
boltons                   23.0.0          py310h06a4308_0  
brotlipy                  0.7.0           py310h7f8727e_1002  
bzip2                     1.0.8                h7b6447c_0  
ca-certificates           2023.5.7             hbcca054_0    conda-forge
certifi                   2023.5.7           pyhd8ed1ab_0    conda-forge
cffi                      1.15.1          py310h5eee18b_3  
charset-normalizer        2.0.4              pyhd3eb1b0_0  
conda                     23.5.0          py310hff52083_1    conda-forge
conda-content-trust       0.1.3           py310h06a4308_0  
conda-package-handling    2.1.0           py310h06a4308_0  
conda-package-streaming   0.8.0           py310h06a4308_0  
cry

## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Prepare data

You don't need to create a training script for automated machine learning, but you do need to load the training data.

In this case, you'll use a dataset containing details of diabetes patients.

To pass a dataset as an input to an automated machine learning job, the data must be in tabular form and include a target column. For the data to be interpreted as a tabular dataset, the input dataset must be a MLTable.

In [4]:
import pandas as pd

# Specify the path to the CSV file
csv_file_path = 'data/diabetes.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

In [5]:
# Display the first few rows of the DataFrame
df.head()

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0


## MLTable data creation

In [8]:
import mltable

paths = [
    {'file': './data/diabetes.csv'}
]

train_table = mltable.from_delimited_files(paths)
train_table.save('./data')


paths:
- file: file:///mnt/batch/tasks/shared/LS_root/mounts/clusters/tj-compute-instance/code/Users/taseen.junaid/Azure-AutoML-Custom-Deployment/data/diabetes.csv
transformations:
- read_delimited:
    delimiter: ','
    empty_as_string: false
    encoding: utf8
    header: all_files_same_headers
    include_path_column: false
    infer_column_types: true
    partition_size: 20971520
    path_column: Path
    support_multi_line: false
type: mltable

## Data loading

In [9]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="./data")

# **Put your compute instance/cluster name**

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named tj-compute-cluster
- Sets Diabetic as the target column
- Sets accuracy as the primary metric
- Times out after 60 minutes of total training time
- No model will be trained with the LogisticRegression algorithm

In [10]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="tj-compute-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)
     

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

Note: This may take some time!

In [12]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading data (0.52 MBs): 100%|██████████| 518385/518385 [00:00<00:00, 14565592.66it/s]




Monitor your job at https://ml.azure.com/runs/patient_stone_f7q7hjzq2j?wsid=/subscriptions/261f30e1-c860-4df8-9a71-1883dfa4caa8/resourcegroups/ai/workspaces/aai&tid=c435b59a-90d0-4884-9465-b5ad0aba32e2


While the job is running, you can monitor it in the Studio.